In [3]:
import os
import mido
from pathlib import Path
from mido import MidiFile, MidiTrack, merge_tracks

In [4]:
mid_file = 'Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep.mid'

In [5]:
mid = mido.MidiFile("../data/raw/" + mid_file)

## COMBINE MIDI TRACKS

In [6]:
def combine_midi_tracks(mid, Returns_path="../data/processed/"):
    if not isinstance(mid, MidiFile):
        raise ValueError("Expected a MidiFile object as input")
    
    combined_midi = MidiFile(ticks_per_beat=mid.ticks_per_beat)

    combined_midi.tracks = [merge_tracks(mid.tracks)]

    Returns_file = os.path.join(Returns_path, Path(mid.filename).stem + "_merged.mid")
    combined_midi.save(Returns_file)

In [7]:
def split_midi_file(mid, Returns_path="../data/processed/"):
    if not isinstance(mid, MidiFile):
        raise ValueError("Expected a MidiFile object as input")
    
    split_midi = MidiFile()
    split_midi.ticks_per_beat = mid.ticks_per_beat
    for msg in mid: 
        if not msg.is_meta:
            split_midi.tracks.append([msg])

In [8]:
print(mid.ticks_per_beat)
print(type(mid))
print(len(mid.tracks))
combine_midi_tracks(mid)

## Split by bars

In [9]:
mid_file = "Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged.mid"
mid_unify = mido.MidiFile("../data/processed/" + mid_file)

In [10]:
BARS_TO_EXTRACT = 4

In [11]:
def save_bar_messages(message, bar_number, ticks_per_beat, filename, Returns_path="../data/processed/"):
    bar_midi = MidiFile(ticks_per_beat=ticks_per_beat)
    bar_track = MidiTrack()
    bar_midi.tracks.append(bar_track)
    for msg in message:
        msg.time = msg.time # int(msg.time)
        bar_track.append(msg)
    Returns_file = os.path.join(Returns_path, f"{filename}_{bar_number}.mid")
    bar_midi.save(Returns_file)
    print(f"Bar {bar_number} saved to {Returns_file}")

In [12]:
metadata_info = [msg for msg in mid_unify if msg.is_meta]
metadata_info[:-1]
metadata_info[-1]

In [24]:
# BUENO
def split_midi_by_bar(mid, bars_to_extract = 4, path="data/processed"):
    filename = Path(mid.filename).stem

    ticks_per_beat = mid.ticks_per_beat
    numerator_time_signature = [msg.numerator for msg in mid if msg.type == 'time_signature'][0]
    ticks_per_bar = ticks_per_beat * numerator_time_signature
    ticks_to_extract = ticks_per_bar * bars_to_extract

    current_bar = 0
    current_ticks = 0
    metadata_info = [msg for msg in mid if msg.is_meta]
    bar_messages = []
    chord_messages = []

    for track in mid.tracks:
        for msg in track:
            if msg.time == 0:
                chord_messages.append(msg)
            else: 
                chord_messages.append(msg)
                # CASE 1: the chord is of the next section
                if current_ticks + msg.time >= ticks_to_extract:
                    current_bar += 1
                    current_ticks = 0
                    bar_messages.append(metadata_info[-1])
                    save_bar_messages(bar_messages, current_bar, ticks_per_beat, filename)
                    bar_messages = metadata_info[:-1]
                    bar_messages.extend(chord_messages)
                    chord_messages = []
                # CASE 2: the chord is of the current section
                else:
                    current_ticks += msg.time
                    bar_messages.extend(chord_messages)
                    chord_messages = []

In [ ]:
# IMPROVED VERSION
def split_midi_by_bar(mid, bars_to_extract = 4, path="data/processed"):
    filename = Path(mid.filename).stem

    ticks_per_beat = mid.ticks_per_beat
    numerator_time_signature = next((msg.numerator for msg in mid if msg.type == 'time_signature'), 4)
    ticks_per_bar = ticks_per_beat * numerator_time_signature
    ticks_to_extract = ticks_per_bar * bars_to_extract

    current_bar = 0
    current_ticks = 0
    metadata_info = [msg for msg in mid if msg.is_meta]
    bar_messages = []
    chord_messages = []

    for track in mid.tracks:
        for msg in track:
            chord_messages.append(msg)

            if msg.time > 0: 
                current_ticks += msg.time

                if current_ticks >= ticks_to_extract:
                    current_bar += 1
                    current_ticks -= ticks_to_extract

                    # Save the current bar messages
                    bar_messages.extend(metadata_info)
                    bar_messages.append(metadata_info[-1])
                    save_bar_messages(bar_messages, current_bar, ticks_per_beat, filename)

                    # Prepare for the next bar
                    bar_messages = metadata_info[:-1]
                    chord_messages.clear()

    # Save remaining messages if any
    if chord_messages:
        current_bar += 1
        bar_messages.extend(chord_messages)
        save_bar_messages(bar_messages, current_bar, ticks_per_beat, filename)

In [26]:
mid = mido.MidiFile("../data/processed/" + mid_file)
split_midi_by_bar(mid, BARS_TO_EXTRACT)

In [16]:
mid = mido.MidiFile("../data/processed/" + mid_file)
filename = Path(mid.filename).stem

ticks_per_beat = mid.ticks_per_beat
numerator_time_signature = [msg.numerator for msg in mid_unify if msg.type == 'time_signature'][0]
ticks_per_bar = ticks_per_beat * numerator_time_signature
ticks_to_extract = ticks_per_bar * 4

current_bar = 0
current_ticks = 0
metadata_info = [msg for msg in mid_unify if msg.is_meta]
bar_messages = []
chord_messages = []

for track in mid.tracks:
    for msg in track:
        if msg.time == 0:
            chord_messages.append(msg)
        else: 
            chord_messages.append(msg)
            # CASE 1: the chord is of the next section
            if current_ticks + msg.time >= ticks_to_extract:
                current_bar += 1
                current_ticks = 0
                bar_messages.append(metadata_info[-1])
                save_bar_messages(bar_messages, current_bar, ticks_per_beat, filename)
                bar_messages = metadata_info[:-1]
                bar_messages.extend(chord_messages)
                chord_messages = []
            # CASE 2: the chord is of the current section
            else:
                current_ticks += msg.time
                bar_messages.extend(chord_messages)
                chord_messages = []

In [17]:
mid_test_1 = mido.MidiFile("../data/processed/" + "Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_1.mid")
mid_test_1

In [18]:
mid_test_2 = mido.MidiFile("../data/processed/" + "Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_2.mid")
mid_test_2

In [19]:
mid_test_14 = mido.MidiFile("../data/processed/" + "Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_14.mid")
mid_test_14